In [1]:
import cv2
import mediapipe as mp
import pickle
import numpy as np

In [3]:
left_model_filename = r'Models\left_model.p'
right_model_filename = r'Models\right_model.p'
pose_model_filename = r'Models\pose_model.p'

# Function to load a model from a pickle file
def load_model(filename):
    with open(filename, 'rb') as f:
        model_data = pickle.load(f)
        return model_data['model']

# Loading the pre-trained models
left_model = load_model(left_model_filename)
right_model = load_model(right_model_filename)
pose_model = load_model(pose_model_filename)

print("Models loaded successfully.")

Models loaded successfully.


In [ ]:
# Initialize Mediapipe hands and pose solutions
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose

hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)
pose = mp_pose.Pose(
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

# Define coordinate axes for angle calculations
axes = {
    "x": np.array([1, 0, 0]),
    "-x": np.array([-1, 0, 0]),
    "y": np.array([0, 1, 0]),
    "-y": np.array([0, -1, 0]),
    "z": np.array([0, 0, 1]),
    "-z": np.array([0, 0, -1]),
}

In [ ]:
# Define coordinate axes for angle calculations

def calculate_angle1(vec1, vec2):
    """
    Returns the cosine of the angle between vec1 and vec2.
    """
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    # Prevent division by zero
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0
    cosine_angle = dot_product / (norm_vec1 * norm_vec2)
    return cosine_angle

def angle_between_vectors(v1, v2):
    """
    Returns the angle in degrees between two vectors v1 and v2.
    """
    dot_prod = np.dot(v1, v2)
    mag_v1 = np.linalg.norm(v1)
    mag_v2 = np.linalg.norm(v2)
    # Clip cosine to avoid numerical issues
    cos_theta = np.clip(dot_prod / (mag_v1 * mag_v2), -1.0, 1.0)
    return np.degrees(np.arccos(cos_theta))

def get_coordinates_safe(landmarks, index):
    """
    Returns the x, y, z coordinates of a landmark; returns [-1, -1, -1] if not found.
    """
    try:
        return np.array([landmarks[index].x, landmarks[index].y, landmarks[index].z])
    except IndexError:
        return np.array([-1, -1, -1])

def get_palm_orientation(normal):
    """
    Determines which axis the palm's normal vector is closest to.
    """
    angles = {axis: angle_between_vectors(normal, direction) for axis, direction in axes.items()}
    # Return the axis with the smallest angle
    best_match_axis = min(angles, key=angles.get)
    return best_match_axis

def extract_features(hand_landmarks, pose_landmarks=None):
    """
    Gathers features from hand landmarks (finger angles, normal vector angles, distances to nose).
    If pose_landmarks are provided, extracts wrist-nose distances as well.
    """
    # Pairs of landmarks for finger angles
    hand_pairs = [
        (1, 3),   # Thumb
        (6, 8),   # Index finger
        (10, 12), # Middle finger
        (14, 16), # Ring finger
        (18, 20), # Pinky finger
        (0, 9)    # Palm direction
    ]
    
    features = []
    # Compute angles for finger segments
    for pair in hand_pairs:
        lm1, lm2 = hand_landmarks[pair[0]], hand_landmarks[pair[1]]
        vec = np.array([lm2.x - lm1.x, lm2.y - lm1.y, lm2.z - lm1.z])
        
        angle_x = calculate_angle1(vec, np.array([1, 0, 0]))
        angle_y = calculate_angle1(vec, np.array([0, 1, 0]))
        angle_z = calculate_angle1(vec, np.array([0, 0, 1]))
        
        features.extend([angle_x, angle_y, angle_z])
    
    # Vectors for palm normal
    vector_0_to_5 = get_coordinates_safe(hand_landmarks, 5) - get_coordinates_safe(hand_landmarks, 0)
    vector_0_to_17 = get_coordinates_safe(hand_landmarks, 17) - get_coordinates_safe(hand_landmarks, 0)
    normal_vector = np.cross(vector_0_to_5, vector_0_to_17)
    
    # Angles between normal vector and coordinate axes
    normal_angle_x = calculate_angle1(normal_vector, np.array([1, 0, 0]))
    normal_angle_y = calculate_angle1(normal_vector, np.array([0, 1, 0]))
    normal_angle_z = calculate_angle1(normal_vector, np.array([0, 0, 1]))
    features.extend([normal_angle_x, normal_angle_y, normal_angle_z])
    
    # Distance from wrist (landmark 0) to nose (pose landmark 0)
    if pose_landmarks is not None:
        nose_landmark = get_coordinates_safe(pose_landmarks, 0)
        wrist_landmark = get_coordinates_safe(hand_landmarks, 0)
        distance_x = abs(nose_landmark[0] - wrist_landmark[0])
        distance_y = abs(nose_landmark[1] - wrist_landmark[1])
        features.extend([distance_x, distance_y])
    
    return features

In [6]:

def calculate_normal_safe(p1, p2, p3):
    """
    Calculates the normal of the plane formed by three points (p1, p2, p3).
    Returns [-1, -1, -1] if any point is invalid.
    """
    if any(np.array_equal(pt, [-1, -1, -1]) for pt in [p1, p2, p3]):
        return np.array([-1, -1, -1])
    return calculate_normal(p1, p2, p3)

def calculate_angle2(p1, p2, p3):
    """
    Calculates the cosine of the angle at p2 formed by the vectors p1->p2 and p3->p2.
    """
    v1 = p1 - p2
    v2 = p3 - p2
    if np.linalg.norm(v1) == 0 or np.linalg.norm(v2) == 0:
        return -1
    cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    return cos_theta

def calculate_normal(p1, p2, p3):
    """
    Calculates and returns the normalized cross-product-based normal of the plane 
    formed by points p1, p2, and p3.
    """
    v1 = p2 - p1
    v2 = p3 - p1
    normal = np.cross(v1, v2)
    norm_val = np.linalg.norm(normal)
    if norm_val == 0:
        return np.array([-1, -1, -1])
    return normal / norm_val

def calculate_normal_angles(normal):
    """
    Returns a list of dot products (cosine values) between the normal vector 
    and each axis in the identity matrix (x, y, z).
    """
    cos_values = []
    for axis in np.eye(3):
        cos_values.append(np.dot(normal, axis))
    return cos_values

def calculate_xy_distance(p1, p2):
    """
    Calculates and returns the absolute differences in x and y coordinates 
    between two points p1 and p2.
    """
    x_distance = abs(p1[0] - p2[0])
    y_distance = abs(p1[1] - p2[1])
    return x_distance, y_distance

def extract_pose_features(image, landmarks):
    """
    Extracts pose features related to angles formed by specific joints, 
    normal vectors, and distances between landmarks in a pose.
    
    Parameters:
        image      - The original image frame (unused here, but can be useful for display/logging).
        landmarks  - List or array of pose landmarks from a Mediapipe result.
        
    Returns:
        A list of feature values including:
          - Cosine of joint angles.
          - Normal vector angles with global axes.
          - Distance between certain landmarks.
    """
    # Define relevant point sets for angles and normals
    points_sets = {
        "angle_11_12_14": (get_coordinates_safe(landmarks, 11),
                           get_coordinates_safe(landmarks, 12),
                           get_coordinates_safe(landmarks, 14)),
        "angle_12_14_16": (get_coordinates_safe(landmarks, 12),
                           get_coordinates_safe(landmarks, 11),
                           get_coordinates_safe(landmarks, 13)),
        "angle_11_13_15": (get_coordinates_safe(landmarks, 11),
                           get_coordinates_safe(landmarks, 13),
                           get_coordinates_safe(landmarks, 15)),
        "angle_13_15_17": (get_coordinates_safe(landmarks, 12),
                           get_coordinates_safe(landmarks, 14),
                           get_coordinates_safe(landmarks, 16)),
        "normal_1":       (get_coordinates_safe(landmarks, 15),
                           get_coordinates_safe(landmarks, 17),
                           get_coordinates_safe(landmarks, 19)),
        "normal_2":       (get_coordinates_safe(landmarks, 16),
                           get_coordinates_safe(landmarks, 18),
                           get_coordinates_safe(landmarks, 20))
    }

    angles = []
    # Calculate angles for specified joints
    for key, (p1, p2, p3) in points_sets.items():
        if key.startswith("angle"):
            angles.append(calculate_angle2(p1, p2, p3))

    # Calculate normals and their angles with the axes
    for key, (p1, p2, p3) in points_sets.items():
        if key.startswith("normal"):
            normal = calculate_normal_safe(p1, p2, p3)
            if np.array_equal(normal, [-1, -1, -1]):
                angles.extend([-1, -1, -1])
            else:
                angles.extend(calculate_normal_angles(normal))

    # Distance between wrists (landmarks 15 and 16)
    p15 = get_coordinates_safe(landmarks, 15)
    p16 = get_coordinates_safe(landmarks, 16)
    x_distance, y_distance = calculate_xy_distance(p15, p16)
    angles.extend([x_distance, y_distance])

    return angles

def calulating_percentage(avg, all_classes):
    """
    Given a list of average probabilities (avg) and corresponding class labels (all_classes),
    returns a list of percentage values adjusted by class-specific thresholds.
    """
    individual_threshold = {
        "sun": 0.9, "help": 0.9, "teacher": 0.9, "support": 0.9,
        "paper": 0.9, "love": 0.9, "dance": 0.9, "water": 0.9,
        "accident": 0.9, "yes": 0.9, "thick": 0.9, "high": 0.9,
        "poor": 0.9, "i": 0.9, "my": 0.9, "important_1": 0.9,
        "important_2": 0.9, "deaf": 0.9, "winner": 0.9, "eat": 0.9,
        "pizza": 0.9, "go": 0.9, "isl": 0.9, "friend": 0.9,
        "school": 0.9, "deep": 0.9, "loud": 0.9, "flat": 0.9,
        "slow": 0.9, "sad": 0.9, "soft": 0.9, "happy": 0.9,
        "poot": 0.9, "quiet": 0.9, "book": 0.9, "woman": 0.9
    }

    threshold_percentage = []
    for score, cls in zip(avg, all_classes):
        # Use class-specific threshold to adjust the final percentage
        threshold_val = individual_threshold.get(cls.lower(), 1.0)
        threshold_percentage.append(score * 100 / threshold_val)

    return threshold_percentage

# Helper function to get landmark coordinates
def get_coordinates_safe(landmarks, index):
    """
    Retrieves the x, y, z coordinates of a given landmark index; 
    returns [-1, -1, -1] if the index is invalid.
    """
    try:
        return np.array([landmarks[index].x, landmarks[index].y, landmarks[index].z])
    except (IndexError, AttributeError):
        return np.array([-1, -1, -1])

In [ ]:
# Track the number of frames, accumulated probabilities, and final drawn prediction
frame_count = 0
accumulated_probs = None
final_prediction_text = ""

cap = cv2.VideoCapture(0)

while True:
    # Capture a frame
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the frame for Mediapipe
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    results_pose = pose.process(image_rgb)

    # Initialize prediction outputs
    left_prediction, right_prediction, pose_prediction = None, None, None
    left_probs, right_probs, pose_probs = None, None, None

    # Detect and process hand landmarks (up to two hands)
    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            label = handedness.classification[0].label
            features = extract_features(
                hand_landmarks.landmark,
                results_pose.pose_landmarks.landmark if results_pose.pose_landmarks else []
            )

            # Classify left or right hand
            if label == 'Left':
                left_prediction = left_model.predict([features])[0]
                left_probs = left_model.predict_proba([features])[0]
            elif label == 'Right':
                right_prediction = right_model.predict([features])[0]
                right_probs = right_model.predict_proba([features])[0]

            # Draw hand landmarks on the frame
            mp.solutions.drawing_utils.draw_landmarks(
                frame, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS
            )

    # Detect and process pose landmarks
    if results_pose.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(
            frame, results_pose.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS
        )
        pose_landmarks = results_pose.pose_landmarks.landmark
        pose_features = extract_pose_features(frame, pose_landmarks)
        pose_prediction = pose_model.predict([pose_features])[0]
        pose_probs = pose_model.predict_proba([pose_features])[0]

    # Gather all class labels used by the three models
    all_classes = sorted(
        set(left_model.classes_).union(
            set(right_model.classes_)
        ).union(
            set(pose_model.classes_)
        )
    )

    # Align probabilities with the master list of classes
    left_probs_aligned = np.zeros(len(all_classes))
    right_probs_aligned = np.zeros(len(all_classes))
    pose_probs_aligned = np.zeros(len(all_classes))

    if left_probs is not None:
        left_dict = dict(zip(left_model.classes_, left_probs))
        left_probs_aligned = np.array([left_dict.get(cls, 0) for cls in all_classes]) * 100

    if right_probs is not None:
        right_dict = dict(zip(right_model.classes_, right_probs))
        right_probs_aligned = np.array([right_dict.get(cls, 0) for cls in all_classes]) * 100

    if pose_probs is not None:
        pose_dict = dict(zip(pose_model.classes_, pose_probs))
        pose_probs_aligned = np.array([pose_dict.get(cls, 0) for cls in all_classes]) * 100

    # Determine the number of available sources (hand(s)/pose)
    num_sources = sum(prob is not None for prob in [left_probs, right_probs, pose_probs])
    # Average the probabilities (divide by 100 × num_sources to keep scale consistent)
    avg = (left_probs_aligned + right_probs_aligned + pose_probs_aligned) / (100 * num_sources if num_sources else 1)

    # Convert these averages into final percentages based on custom thresholds
    avg_probs = calulating_percentage(avg, all_classes)

    if accumulated_probs is None:
        accumulated_probs = np.zeros_like(avg_probs)
    accumulated_probs += avg_probs
    frame_count += 1

    # Updating the final prediction text after a fixed no. of frames (e.g., 5)
    if frame_count == 5:
        max_idx = np.argmax(accumulated_probs)
        max_class = all_classes[max_idx]
        final_prediction_text = f"Final Prediction: {max_class}, Prob: {accumulated_probs[max_idx]:.2f}"
        accumulated_probs = None
        frame_count = 0

    if final_prediction_text:
        cv2.putText(
            frame,
            final_prediction_text,
            (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (255, 0, 0),
            2
        )

    # Visualizing the accumulated probabilities on the screen
    if accumulated_probs is not None:
        y_offset = 60
        for i, prob in enumerate(accumulated_probs):
            cv2.putText(
                frame,
                f"{all_classes[i]}: {prob:.2f}",
                (10, y_offset),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.8,
                (0, 0, 255),
                2
            )
            y_offset += 30

    cv2.imshow("Hand and Pose Tracking", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()